### Lab_03

### STEP 1: GET A BIGGER DATASET

 ### Generate the dataset:


In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random
# Simulate 60 days of data
customers = ['Amazon', 'Walmart', 'Target', 'Costco', 'BestBuy', 'eBay']
data = []
start_date = datetime(2025, 4, 1)
for i in range(1, 61):
    date = start_date + timedelta(days=i)
    for _ in range(random.randint(3, 6)):  # 3–6 sales per day
        data.append({
            'id': random.randint(1000, 9999),
            'customer': random.choice(customers),
            'date': date.date().isoformat(),
            'amount': random.randint(100, 2000),
            'last_updated': (date + timedelta(hours=random.randint(0, 23),
                                              minutes=random.randint(0, 59))).isoformat()
        })
df = pd.DataFrame(data)
df.to_csv('sales_data_large.csv', index=False)
df.head()


,id,customer,date,amount,last_updated
0,8983,Costco,2025-04-02,604,2025-04-02T02:22:00
1,8601,Costco,2025-04-02,416,2025-04-02T02:13:00
2,9833,eBay,2025-04-02,1501,2025-04-02T17:54:00
3,6071,Walmart,2025-04-02,711,2025-04-02T21:45:00
4,2268,Target,2025-04-03,513,2025-04-03T07:27:00


 ###  STEP 2: FULL EXTRACTION

In [2]:
# FULL EXTRACTION
df_full = pd.read_csv("sales_data_large.csv", parse_dates=["last_updated"])
print(f"Pulled {len(df_full)} rows via full extraction.")
df_full.head()


Pulled 285 rows via full extraction.


,id,customer,date,amount,last_updated
0,8983,Costco,2025-04-02,604,2025-04-02 02:22:00
1,8601,Costco,2025-04-02,416,2025-04-02 02:13:00
2,9833,eBay,2025-04-02,1501,2025-04-02 17:54:00
3,6071,Walmart,2025-04-02,711,2025-04-02 21:45:00
4,2268,Target,2025-04-03,513,2025-04-03 07:27:00


### STEP 3: INCREMENTAL EXTRACTION

### Create your tracking file

In [4]:
# Set initial last extraction time (e.g., halfway through the data range)
with open("last_extraction.txt", "w") as f:
    f.write("2025-04-20 12:00:00") 

In [5]:
# INCREMENTAL EXTRACTION
with open("last_extraction.txt", "r") as f:
    last_extraction = f.read().strip()
df = pd.read_csv("sales_data_large.csv", parse_dates=["last_updated"])
last_extraction_time = pd.to_datetime(last_extraction)
df_incremental = df[df['last_updated'] > last_extraction_time]
print(f"Pulled {len(df_incremental)} new/updated rows since {last_extraction}.")
df_incremental.head()


Pulled 199 new/updated rows since 2025-04-20 12:00:00.


,id,customer,date,amount,last_updated
86,3112,Costco,2025-04-21,417,2025-04-21 04:46:00
87,3627,eBay,2025-04-21,1739,2025-04-21 18:21:00
88,7261,Walmart,2025-04-21,177,2025-04-21 14:15:00
89,3724,Amazon,2025-04-21,1316,2025-04-21 20:33:00
90,4571,Amazon,2025-04-21,1563,2025-04-21 18:06:00


### Update the last_extraction.txt

In [6]:
# Get the most recent update
new_checkpoint = df['last_updated'].max()
# Save it
with open("last_extraction.txt", "w") as f:
    f.write(new_checkpoint.isoformat())
print(f"Updated last_extraction.txt to {new_checkpoint}")

Updated last_extraction.txt to 2025-05-31 22:25:00
